<a href="https://colab.research.google.com/github/Isshoo/palm-recognition-model/blob/main/Palm_Recognition_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tensorflow opencv-python matplotlib numpy

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2

In [ ]:
# Mount Google Drive (if your dataset is there)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir -p "/content/dataset"

In [ ]:
!cp -r "/content/drive/MyDrive/palm_dataset/" "/content/dataset/"

In [ ]:
# Configuration
DATASET_PATH = "/content/drive/MyDrive/gabungan_palm/"
INPUT_SIZE = (256, 256)  # Reduced size for faster processing with small dataset
BATCH_SIZE = 8  # Smaller batch size for limited data
INITIAL_LR = 1e-4
EPOCHS = 100  # More epochs for small dataset with augmentation
SEED = 42

In [ ]:
# Enhanced random seed setup
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

In [ ]:
# Performance optimizations
tf.config.optimizer.set_experimental_options({'auto_mixed_precision': True})
# Enable memory growth to prevent GPU memory issues
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Enable XLA compilation for faster execution
tf.config.optimizer.set_jit(True)

In [ ]:
# Check if dataset exists
if not os.path.exists(DATASET_PATH):
    print(f"ERROR: Dataset path {DATASET_PATH} not found!")
    print("Please check your dataset path.")
else:
    CLASS_NAMES = sorted([d for d in os.listdir(DATASET_PATH)
                         if os.path.isdir(os.path.join(DATASET_PATH, d))])
    print("Class names:", CLASS_NAMES)
    print(f"Number of classes: {len(CLASS_NAMES)}")

    # Check images per class
    for class_name in CLASS_NAMES:
        class_path = os.path.join(DATASET_PATH, class_name)
        image_files = [f for f in os.listdir(class_path)
                      if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        print(f"Class '{class_name}': {len(image_files)} images")

In [ ]:
def enhanced_palm_preprocessing(image_path, visualize=False):
    """
    Enhanced preprocessing specifically for palm line detection
    Optimized for rotated palm images without fingers
    """
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Could not read image: {image_path}")

    steps = []
    titles = []

    # Apply CLAHE to L channel for better contrast
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_img = clahe.apply(img)
    steps.append(clahe_img)
    titles.append("Clahe")

    # Denoising with Non-local Means - adjusted parameters
    denoised = cv2.fastNlMeansDenoising(clahe_img, None, h=10, templateWindowSize=7, searchWindowSize=21)
    steps.append(denoised)
    titles.append("Denoised")

    # Ridge Enhancement using DoG (Difference of Gaussians)
    gauss1 = cv2.GaussianBlur(denoised, (0, 0), sigmaX=0.5)
    gauss2 = cv2.GaussianBlur(denoised, (0, 0), sigmaX=1.5)
    dog = gauss1 - gauss2
    dog = cv2.normalize(dog, None, 0, 255, cv2.NORM_MINMAX)
    steps.append(dog)
    titles.append("DoG Enhancement")

    # Resize with high-quality interpolation
    resized = cv2.resize(dog, INPUT_SIZE, interpolation=cv2.INTER_LANCZOS4)
    steps.append(resized)
    titles.append("Resized")

    # Normalize to [0, 1] range
    normalized = resized.astype(np.float32) / 255.0
    steps.append(normalized)
    titles.append("Normalized")

    if visualize:
        plt.figure(figsize=(18, 12))
        for i, (step_img, title) in enumerate(zip(steps, titles), 1):
            plt.subplot(2, 5, i)
            if len(step_img.shape) == 2:
                plt.imshow(step_img, cmap='gray')
            else:
                plt.imshow(step_img)
            plt.title(title, fontsize=10)
            plt.axis('off')
        plt.tight_layout()
        plt.show()
        return None

    return normalized

In [ ]:
# Test preprocessing dengan sample image
if 'CLASS_NAMES' in locals() and CLASS_NAMES:
    sample_class = CLASS_NAMES[0]
    class_path = os.path.join(DATASET_PATH, sample_class)
    if os.path.exists(class_path):
        sample_images = [f for f in os.listdir(class_path)
                        if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        if sample_images:
            sample_path = os.path.join(class_path, sample_images[50])
            print(f"Testing preprocessing with: {sample_path}")
            enhanced_palm_preprocessing(sample_path, visualize=True)

In [ ]:
def load_dataset(dataset_path, class_names, test_size=0.1, val_size=0.1):
    """Load and split dataset into train, validation and test sets"""
    images = []
    labels = []

    for class_idx, class_name in enumerate(class_names):
        class_path = os.path.join(dataset_path, class_name)
        image_files = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        print(f"Processing {len(image_files)} images for class {class_name}...")
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            try:
                processed_img = enhanced_palm_preprocessing(image_path)
                images.append(processed_img)
                labels.append(class_idx)
            except Exception as e:
                print(f"Error processing {image_path}: {e}")

    # Convert to numpy arrays
    images = np.array(images)
    labels = np.array(labels)

    # One-hot encode labels
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    labels = to_categorical(labels, num_classes=len(class_names))

    # Split into train, validation and test
    X_train, X_test, y_train, y_test = train_test_split(
        images, labels, test_size=test_size, random_state=SEED, stratify=labels
    )

    # Further split train into train and validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=val_size, random_state=SEED, stratify=y_train
    )

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)

In [ ]:
from tensorflow.keras import layers, models

def create_enhanced_model(input_shape, num_classes):
    """Create enhanced model with more regularization"""

    model = models.Sequential([
        layers.Input(shape=input_shape),  # 1 channel karena grayscale

        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D(),

        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(),

        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D(),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')  # output layer
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
# Load dataset
print("Loading dataset...")
(X_train, y_train), (X_val, y_val), (X_test, y_test) = load_dataset(DATASET_PATH, CLASS_NAMES)

print(f"\nDataset shapes:")
print(f"Train: {X_train.shape}, {y_train.shape}")
print(f"Validation: {X_val.shape}, {y_val.shape}")
print(f"Test: {X_test.shape}, {y_test.shape}")

In [ ]:
# Create enhanced model
print("\nCreating enhanced model...")
model = create_enhanced_model((INPUT_SIZE[0], INPUT_SIZE[1], 1), len(CLASS_NAMES))
model.summary()

In [ ]:
# Enhanced callbacks
callbacks = [
    ModelCheckpoint(
        'best_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=1
    )
]

In [ ]:
# Enhanced training with class weights
print("\nTraining enhanced model...")
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# Load best model
model.load_weights('debug_model.keras')

In [ ]:
# Evaluate
print("\n" + "="*50)
print("EVALUATION")
print("="*50)

train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)

print(f"\nResults:")
print(f"Train Accuracy: {train_acc*100:.2f}%")
print(f"Validation Accuracy: {val_acc*100:.2f}%")
print(f"Test Accuracy: {test_acc*100:.2f}%")

# Predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes,
                          target_names=CLASS_NAMES, zero_division=0))

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Show some predictions
print("\n" + "="*50)
print("SAMPLE PREDICTIONS")
print("="*50)

# Show first few test predictions
for i in range(min(5, len(X_test))):
    true_class = CLASS_NAMES[y_true_classes[i]]
    pred_class = CLASS_NAMES[y_pred_classes[i]]
    confidence = y_pred[i][y_pred_classes[i]]

    print(f"Sample {i+1}: True={true_class}, Pred={pred_class}, Confidence={confidence:.3f}")

    # Show image
    plt.figure(figsize=(4, 4))
    plt.imshow(X_test[i].squeeze(), cmap='gray')
    plt.title(f"True: {true_class}\nPred: {pred_class} ({confidence:.3f})")
    plt.axis('off')
    plt.show()

In [ ]:
# Save final model
model.save('final_palm_recognition_model.keras')
print("\nModel saved as 'final_palm_recognition_model.keras'")

In [ ]:
# Convert to TFLite (optional)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('palm_line_detection.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model saved in H5 and TFLite formats!")

In [ ]:
"""
## 9. Inference Example
"""

def predict_image(image_path, model, class_names):
    """Make prediction on a single image"""
    # Preprocess image
    processed_img = preprocessing_image(image_path)

    # Add batch dimension
    input_img = np.expand_dims(processed_img, axis=0)

    # Make prediction
    predictions = model.predict(input_img)
    predicted_class = np.argmax(predictions)
    confidence = np.max(predictions)

    # Display results
    plt.figure(figsize=(8, 4))

    # Original image
    original_img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    plt.subplot(1, 2, 1)
    plt.imshow(original_img)
    plt.title("Original Image")
    plt.axis('off')

    # Processed image with prediction
    plt.subplot(1, 2, 2)
    plt.imshow(processed_img)
    plt.title(f"Predicted: {class_names[predicted_class]}\nConfidence: {confidence:.2f}")
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    return class_names[predicted_class], confidence

In [ ]:
# Test prediction
sample_image = os.path.join(DATASET_PATH, CLASS_NAMES[1], os.listdir(os.path.join(DATASET_PATH, CLASS_NAMES[1]))[14])
pred_class, confidence = predict_image(sample_image, model, CLASS_NAMES)
print(f"Predicted class: {pred_class} with confidence {confidence:.2f}")